# Tipos Algébricos

Para além das listas definidar indutivamente, podemos definir outros tipos de dados. Um exemplo é o tipo árvore binária.

## Árvores binárias
Uma árvore binária é um tipo de dado que pode ser definido indutivamente como:

In [1]:
type 'a tree = Empty | Node of 'a * 'a tree * 'a tree

type 'a tree = Empty | Node of 'a * 'a tree * 'a tree


Um exemplo de uma árvore binária com o seguinte aspeto é:

```
    2
   / \
  1   3
```


In [2]:
let t0 = Node(2, Node(1, Empty, Empty), Node(3, Empty, Empty))

val t0 : int tree = Node (2, Node (1, Empty, Empty), Node (3, Empty, Empty))


Para inserir um elemento numa árvore binária em OCaml vamos gerar a árvore binária que corresponder à inserção do elemento. Vamos fazê-lo indutivamente na estrutura da árvore. Há muitas alternativas para inserir um elemento numa árvore binária. Uma delas é a seguinte:


In [3]:
let rec insert x t = 
  match t with 
  | Empty -> Node(x, Empty, Empty)
  | Node(y,l,r) -> Node(x,t,Empty)

let _ = assert (insert 0 t0 = Node (0, Node (2, Node (1, Empty, Empty), Node (3, Empty, Empty)), Empty))


val insert : 'a -> 'a tree -> 'a tree = <fun>


- : unit = ()



## Árvores binárias de pesquisa
Uma árvore binária de pesquisa é uma árvore binária com a seguinte propriedade: para cada nó, o valor do nó é maior que todos os valores da subárvore esquerda e menor que todos os valores da subárvore direita. Adicionalmente a subárvore esquerda e a subárvore direita também são árvores binárias de pesquisa. 

A inserção de um elemento numa árvore binária de pesquisa de forma a manter a propriedade de árvore binária de pesquisa é feita da seguinte forma:

In [8]:

let rec insert_bst x t = 
  match t with 
  | Empty -> Node(x, Empty, Empty)
  | Node(y,l,r) -> if x < y then Node(y,insert_bst x l,r) else Node(y,l, insert_bst x r)

let t1 = insert_bst 0 t0
let t2 = Node(4, t0, Empty)
let _ = assert(t1 = Node (2, Node (1, Node (0, Empty, Empty), Empty), Node (3, Empty, Empty)))


val insert_bst : 'a -> 'a tree -> 'a tree = <fun>


val t1 : int tree =
  Node (2, Node (1, Node (0, Empty, Empty), Empty), Node (3, Empty, Empty))


val t2 : int tree =
  Node (4, Node (2, Node (1, Empty, Empty), Node (3, Empty, Empty)), Empty)


- : unit = ()


Vamos definir uma função auxiliar `remove_min` que remove o nó com o menor valor de uma árvore binária de pesquisa. Esta função vai ser útil para remover um nó da árvore binária de pesquisa. A função `remove_min` vai devolver a árvore binária de pesquisa sem o nó com o menor valor.

In [5]:
(** [remove_min t] denotes the tree removing the minimum value from [t]
    pre: [t <> Empty] *)
let rec remove_min t = 
  match t with 
  | Empty -> failwith "Precondition violated!"
  | Node(x,Empty,r) -> r
  | Node(x,l,r) -> Node(x, remove_min l, r)

(** [min_tree t] denotes the smaller value in [t]
    pre: [t <> Empty] *)
let rec min_tree t = 
  match t with 
  | Empty -> failwith "Precondition violated!"
  | Node(x,Empty,r) -> x
  | Node(x,l,r) -> min_tree l

(** [min_tree t] denotes the larger value in [t]
    pre: [t <> Empty] *)
let rec max_tree t = 
  match t with 
  | Empty -> failwith "Precondition violated!"
  | Node(x,_,Empty) -> x
  | Node(_,_,r) -> max_tree r

val remove_min : 'a tree -> 'a tree = <fun>


val min_tree : 'a tree -> 'a = <fun>


val max_tree : 'a tree -> 'a = <fun>


Dadas estas funções auxiliares, podemos definir uma função que corresponde à prova indutiva da propriedade das árvores binárias de pesquisa. Se considerarmos que a chamada recursiva para a subárvore esquerda e para a subárvore direita corresponde à hipótese de indução (a propriedade para uma árvore mais pequena), então podemos definir o resultado para a árvore composta verificando se todos os elementos na subárvore esquerda são menores que a raiz e todos os elementos na subárvore direita são maiores que a raiz.

In [ ]:
let rec is_a_bst t = 
  match t with 
  (* Base case *)
  | Empty -> true  

  (* Base case *)
  | Node(_, Empty,Empty) -> true

  (* Inductive case if the left subtree is empty *)
  | Node(x,Empty,r) -> (* by h.i. *) is_a_bst r && min_tree r >= x

  (* Inductive case if the right subtree is empty *)
  | Node(x,l,Empty) -> (* by h.i. *) is_a_bst l && max_tree l < x 

  (* General Inductive case *)
  | Node(x,l,r) -> (* by h.i. *) is_a_bst l && (* by h.i. *) is_a_bst r && max_tree l < x && min_tree r >= x

let _ = assert(is_a_bst t0)
let _ = assert(is_a_bst t1)
let _ = assert(is_a_bst t2)


val is_a_bst : 'a tree -> bool = <fun>


- : unit = ()


- : unit = ()


- : unit = ()


- : int tree = Node (2, Empty, Node (3, Empty, Empty))


...

val remove : 'a -> 'a tree -> 'a tree = <fun>


- : int tree = Node (3, Node (1, Empty, Empty), Empty)


- : int tree =
Node (4, Node (2, Node (1, Empty, Empty), Node (3, Empty, Empty)), Empty)


- : int tree = Node (4, Node (3, Node (1, Empty, Empty), Empty), Empty)


- : bool = true


A remoção de um valor de uma árvore binária de pesquisa é feita procurando o valor e removendo-o. Quando um nó intermédio é removido, é necessário substituí-lo pelo valor mínimo da subárvore direita (ou pelo valor máximo da subárvore esquerda). Isto é feito chamando a função `remove_min` na subárvore direita e substituindo o nó intermédio pelo resultado. A subárvore esquerda permanece inalterada. O resultado é uma nova árvore binária de pesquisa com as mesmas propriedades de antes, mas sem o valor removido.

In [14]:
let rec remove x t = 
  match t with
  | Empty -> Empty
  | Node(y,l,r) -> 
    if x = y then 
      Node(min_tree r, l, remove_min r)
    else 
      if x < y then 
        Node(y, remove x l ,r) 
      else 
        Node(y, l, remove x r)

let _ = assert (remove 2 t0 = Node (3, Node (1, Empty, Empty), Empty))
let _ = assert (remove 2 t2 = Node (4, Node (3, Node (1, Empty, Empty), Empty), Empty))

val remove : 'a -> 'a tree -> 'a tree = <fun>


- : unit = ()


- : unit = ()


- binary trees
- insert in a bst
- remove in a bst (remove_min)
- tree_size
- depth
- pre_order 
- in_order
- post_order
- map
- fold_in_order
- fold_pre_order
- fold_post_order

Uma alternativa à função `remove_min` acima é calcular o valor mínimo ao mesmo tempo que este é removido. A função `remove` é então definida da seguinte forma:

In [16]:
(** pre: [t <> Empty] *)
let rec remove_min t = 
  match t with 
  | Empty -> failwith "There is no minimum"
  | Node(y,Empty,r) -> (y,r)
  | Node(y,l,r) -> let (min,l') = remove_min l in (min, Node(y,l',r))

let rec remove x t = 
  match t with 
  | Empty -> Empty
  | Node(y,l,r) -> 
      if x = y then 
        begin match r with
        | Empty -> l
        | _ -> let (min,r') = remove_min r in Node(min, l, r')
        end
      else if x < y then 
        Node(y, remove x l, r)
      else 
        Node(y, l, remove x r)

let _ = assert (remove 2 t0 = Node (3, Node (1, Empty, Empty), Empty))
let _ = assert (remove 2 t2 = Node (4, Node (3, Node (1, Empty, Empty), Empty), Empty))

val remove_min : 'a tree -> 'a * 'a tree = <fun>


val remove : 'a -> 'a tree -> 'a tree = <fun>


- : unit = ()


- : unit = ()


Outras funções sobre árvores que podemos definir são as seguintes:

In [19]:
let rec tree_size t = 
  match t with
  | Empty -> 0
  | Node(_, l, r) -> 1 + tree_size l + tree_size r

  
let rec tree_depth t = 
  match t with
  | Empty -> 0
  | Node(_,l,r) -> 1 + max (tree_depth l) (tree_depth r)
    
let _ = assert (4 = tree_size t2)
let _ = assert (3 = tree_depth t2)


val tree_size : 'a tree -> int = <fun>


val tree_depth : 'a tree -> int = <fun>


- : unit = ()


- : unit = ()


# Percursos em árvores binárias

Os percursos em árvores binárias são funções que percorrem a árvore de uma forma específica. Existem três tipos de percursos em árvores binárias:

- prefixo (pre-order): visita a raiz, depois a subárvore esquerda e depois a subárvore direita
- infixo (in-order): visita a subárvore esquerda, depois a raiz e depois a subárvore direita
- posfixo (post-order): visita a subárvore esquerda, depois a subárvore direita e depois a raiz

In [25]:
let t = Node(1,Empty,Node(2,Empty,Node (3,Empty,Empty)))
let t3 = Node(0, t, t)

let rec pre_order t = 
  match t with 
  | Empty -> []
  | Node(x,l,r) -> x::(pre_order l)@(pre_order r)

let _ = assert([0;1;2;3;1;2;3] = pre_order t3)

val t : int tree = Node (1, Empty, Node (2, Empty, Node (3, Empty, Empty)))


val t3 : int tree =
  Node (0, Node (1, Empty, Node (2, Empty, Node (3, Empty, Empty))),
   Node (1, Empty, Node (2, Empty, Node (3, Empty, Empty))))


val pre_order : 'a tree -> 'a list = <fun>


- : unit = ()


In [26]:
let rec in_order t = 
  match t with 
  | Empty -> []
  | Node(x,l,r) -> (in_order l)@[x]@(in_order r)

let _ = assert( [1;2;3;0;1;2;3] = in_order t3)

val in_order : 'a tree -> 'a list = <fun>


- : unit = ()


In [28]:
let ot = Node(4,Node(2,Node(1,Empty,Empty),Node(3,Empty,Empty)),Node(6,Node(5,Empty,Empty),Node(7,Empty,Empty)))

let _ = assert( [1; 2; 3; 4; 5; 6; 7] = in_order ot)
let _ = assert( [4; 2; 1; 3; 6; 5; 7] = pre_order ot)

val ot : int tree =
  Node (4, Node (2, Node (1, Empty, Empty), Node (3, Empty, Empty)),
   Node (6, Node (5, Empty, Empty), Node (7, Empty, Empty)))


- : unit = ()


- : unit = ()


In [30]:
let rec post_order t = 
  match t with 
  | Empty -> []
  | Node(x,l,r) -> (post_order l)@(post_order r)@[x]

let _ = assert( [1; 3; 2; 5; 7; 6; 4] = post_order ot)

val post_order : 'a tree -> 'a list = <fun>


- : unit = ()


In [31]:
let rec in_order_rev t = 
  match t with 
  | Empty -> []
  | Node(x,l,r) -> (in_order_rev r)@[x]@(in_order_rev l)

let _ = assert( [7; 6; 5; 4; 3; 2; 1] = in_order_rev ot)

val in_order_rev : 'a tree -> 'a list = <fun>


- : unit = ()


# Funções de ordem superior (Map e Fold)

In [ ]:
let rec map f t = 
  match t with
  | Empty -> Empty
  | Node(x,l,r) -> Node(f x, map f l, map f r)

let _ = map string_of_int ot
let _ = map ((+)1) ot

val map : ('a -> 'b) -> 'a tree -> 'b tree = <fun>


- : string tree =
Node ("4", Node ("2", Node ("1", Empty, Empty), Node ("3", Empty, Empty)),
 Node ("6", Node ("5", Empty, Empty), Node ("7", Empty, Empty)))


- : int tree =
Node (5, Node (3, Node (2, Empty, Empty), Node (4, Empty, Empty)),
 Node (7, Node (6, Empty, Empty), Node (8, Empty, Empty)))


In [ ]:
let map_opt f o = 
  match o with
  | None -> None
  | Some v -> Some (f v)

let _ = map_opt string_of_int (Some 1)
let _ = map_opt string_of_int None
let _ = map_opt ((+)1) (Some 1)
let _ = map_opt ((+)1) None

val map_opt : ('a -> 'b) -> 'a option -> 'b option = <fun>


- : string option = Some "1"


- : string option = None


- : int option = Some 2


- : int option = None


In [ ]:
let rec fold_left f acc l = 
  match l with 
  | [] -> acc
  | x::xs -> fold_left f (f acc x) xs

let _ = fold_left (+) 0 (pre_order ot)

let rec fold_in_order f acc t =  
  match t with
  | Empty -> acc
  | Node(x,l,r) -> let acc_l = fold_in_order f acc l in let acc_root = f acc_l x in fold_in_order f acc_root r

let _ = fold_in_order (+) 0 ot
let _ = fold_in_order (fun acc x -> acc@[x]) [] ot
let filter p t = fold_in_order (fun acc x -> if p x then acc@[x] else acc) [] ot

let _ = filter (fun x -> x mod 2 = 0) ot



val fold_left : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>


- : int = 28


val fold_in_order : ('a -> 'b -> 'a) -> 'a -> 'b tree -> 'a = <fun>


- : int = 28


- : int list = [1; 2; 3; 4; 5; 6; 7]


val filter : (int -> bool) -> 'a -> int list = <fun>


- : int list = [2; 4; 6]


In [ ]:
let rec fold_pre_order f acc t =  
  match t with
  | Empty -> acc
  | Node(x,l,r) -> let acc_root = f acc x in let acc_l = fold_pre_order f acc_root l in fold_pre_order f acc_l r

let _ = fold_pre_order (fun acc x -> acc@[x]) [] ot


val fold_pre_order : ('a -> 'b -> 'a) -> 'a -> 'b tree -> 'a = <fun>


- : int list = [4; 2; 1; 3; 6; 5; 7]
